In [1]:
!pip install torch torchvision scipy datasets jiwer accelerate -U

                                              0.0/172.4 MB ? eta -:--:--
                                              0.1/172.4 MB 2.2 MB/s eta 0:01:20
                                              0.5/172.4 MB 4.7 MB/s eta 0:00:37
                                              0.6/172.4 MB 4.7 MB/s eta 0:00:37
                                              1.0/172.4 MB 5.8 MB/s eta 0:00:30
                                              1.7/172.4 MB 7.2 MB/s eta 0:00:24
                                              2.3/172.4 MB 7.8 MB/s eta 0:00:22
                                              2.8/172.4 MB 8.2 MB/s eta 0:00:21
                                              3.3/172.4 MB 8.4 MB/s eta 0:00:21
                                              3.6/172.4 MB 8.2 MB/s eta 0:00:21
                                              4.0/172.4 MB 8.2 MB/s eta 0:00:21
     -                                        4.5/172.4 MB 8.5 MB/s eta 0:00:20
     -                                        5

In [2]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests
import os
import glob
import pandas as pd

c:\Users\LENOVO\Documents\Satria Data\bdc-team4\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# load image from the IAM database (actually this model is meant to be used on printed text)
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-printed')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-printed')

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
c:\Users\LENOVO\Documents\Satria Data\bdc-team4\venv\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\LENOVO\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-y

In [24]:
train_df = pd.read_csv('retrain-labels.csv', delimiter=';')
test_df = pd.read_csv('DataTestLabelled.csv', delimiter=',')

In [20]:
# from sklearn.model_selection import train_test_split

# train_df, test_df = train_test_split(train_df, test_size=0.2)

In [25]:
test_df = test_df.drop(test_df.columns[0], axis=1)

In [26]:
train_df = train_df.rename(columns={train_df.columns[0]: "file_name", train_df.columns[1]: "text"}, inplace=False)
test_df = test_df.rename(columns={test_df.columns[0]: "text", test_df.columns[1]: "file_name"}, inplace=False)

In [27]:
import torch
from torch.utils.data import Dataset

class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text 
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, 
                                          padding="max_length", 
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [28]:
train_dataset = IAMDataset(root_dir=os.path.join(os.getcwd(),'Retrain-Data/'),
                           df=train_df,
                           processor=processor)
eval_dataset = IAMDataset(root_dir=os.path.join(os.getcwd(),'Test-Data/'),
                           df=test_df,
                           processor=processor)

In [29]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

Number of training examples: 890
Number of validation examples: 100


In [37]:
encoding = train_dataset[1]
for k,v in encoding.items():
  print(k, v.shape)

pixel_values torch.Size([3, 384, 384])
labels torch.Size([128])


In [38]:
image = Image.open(train_dataset.root_dir + train_df['file_name'][1]).convert("RGB")
image

In [39]:
labels = encoding['labels']
labels[labels == -100] = processor.tokenizer.pad_token_id
label_str = processor.decode(labels, skip_special_tokens=True)
print(label_str)

E


In [40]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [46]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    fp16=False, 
    output_dir="./",
    logging_steps=2,
    save_steps=100,
    eval_steps=20,
)

In [42]:
from datasets import load_metric

cer_metric = load_metric("cer")

In [43]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [44]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)
trainer.train()

c:\Users\LENOVO\Documents\Satria Data\bdc-team4\venv\lib\site-packages\transformers\models\trocr\processing_trocr.py:134: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
c:\Users\LENOVO\Documents\Satria Data\bdc-team4\venv\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/336 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [43]:
model.save_pretrained("./")

In [46]:
processor.save_pretrained("./")

In [ ]:
# load image from the IAM database (actually this model is meant to be used on printed text)
processor = TrOCRProcessor.from_pretrained('./tr-ocr-fine-tuned2', local_files_only=True)
model = VisionEncoderDecoderModel.from_pretrained('./tr-ocr-fine-tuned2', local_files_only=True)

In [52]:
generated_plate_numbers = []

In [ ]:
labels_csv = pd.read_csv('DataTestLabelled.csv', delimiter=',')
list_license_plates = labels_csv['Image Labels']
list_file_names = labels_csv['Name of File']

In [53]:
for file in list_file_names:
    path = os.path.join(os.getcwd(), 'Test-Data', file)
    image = Image.open(path)
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    generated_plate_numbers.append(generated_text)

/Users/rsoedarnadi/opt/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [55]:
import re

In [57]:
generated_plate_numbers_ = generated_plate_numbers

In [58]:
for i, platenum in enumerate(generated_plate_numbers_):
   generated_plate_numbers_[i] = re.sub('[^A-Za-z0-9]+', '', platenum)

In [59]:
import numpy as np

In [60]:
def calculate_predicted_accuracy(names, actual_list, predicted_list):
    list_accuracy = []
    for name, actual_plate, predict_plate in zip(names, actual_list, predicted_list):
        accuracy = 0
        num_matches = 0
        if actual_plate == predict_plate:
            accuracy = 1.0
        else:
            for a, p in zip(actual_plate, predict_plate):
                if a == p:
                    num_matches += 1
            accuracy = round((num_matches / len(actual_plate)), 2)
        list_accuracy.append(accuracy)
    list_accuracy = np.asarray(list_accuracy)
    avg_value = np.average(list_accuracy)
    return avg_value

In [61]:
calculate_predicted_accuracy(list_file_names, list_license_plates, generated_plate_numbers_)

0.9330999999999999

In [65]:
labels = pd.Series(generated_plate_numbers_)

In [67]:
test_df = pd.concat([test_df, labels], axis=1)

In [69]:
test_df.to_csv('finetuned trocr testing results.csv')